In [3]:
import os
import sys
import json

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
# root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
sys.path.append(root)


# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from resnet18 import Resnet18
from utils import *


2023-07-26 03:13:17.203446: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 03:13:17.232771: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 03:13:17.670479: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:

with open('conf.json', 'r') as json_file:
    config = json.load(json_file)


In [5]:



print("*" * 20)
print("*" * 20)
if tf.config.list_physical_devices('GPU'):
    tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[config['gpu_index']], 'GPU')
    print("config tensorflow using gpu successfully")
else:
    print("There is no gpu or your tensorflow is not built in with gpu support")
print("*" * 20)
print("*" * 20)


epoch = config['training_params']['epoch']
batch_size = config['training_params']['batch_size']
patience = 2000


********************
********************
config tensorflow using gpu successfully
********************
********************


2023-07-26 03:13:29.386257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-26 03:13:29.403495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-26 03:13:29.403535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [6]:

print('==> Preparing data...')
train_images, train_labels, test_images, test_labels = get_dataset()
data_size = len(train_images)

mean, std = get_mean_and_std(train_images)
train_images = normalize(train_images, mean, std)
test_images = normalize(test_images, mean, std)

train_ds = dataset_generator(train_images, train_labels, batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).\
        batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print('==> Data Ready!')


==> Preparing data...
 44474368/170498071 [======>.......................] - ETA: 5:06

KeyboardInterrupt: 

In [12]:

# Define model
model = Resnet18(input_features= (32, 32, 3), 
                 output_features= 10,
                 lr=config['training_params']['learning_rate'],
                 decay_steps=int(epoch * data_size / batch_size))

# Define framework
tensorflow_framework = TensorflowFramework(model=model, 
                                           data_size= data_size, 
                                           train_ds= train_ds, 
                                           test_ds= test_ds, 
                                           config=config)




In [ ]:



# Initialize variables for early stopping check
best_val_loss = float("inf")
# Number of epochs to wait before stopping training when performance worsens
# already set patience above
waiting = 0

# training with 200 epoch or early stopping
print("*" * 20)
print("*" * 20)
print(f"Training for the total number of epoch {epoch} with batch_size {batch_size} for datasize of {data_size}")
print("*" * 20)
print("*" * 20)
for epoch in range(epoch):
    tensorflow_framework.model.train_loss.reset_states()
    tensorflow_framework.model.train_performance.reset_states()
    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for images, labels in tensorflow_framework.train_ds:
        train_acc, train_loss= tensorflow_framework.fit(images, labels)

    for test_images, test_labels in tensorflow_framework.test_ds:
        test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

    print("Epoch {} - Train Acc: {:.2f} -- Train Loss {} Test Acc {:.2f}  Test Loss {}".format(epoch+1,
                                                                                       train_acc * 100,
                                                                                       train_loss,
                                                                                       test_acc * 100,
                                                                                       test_loss))
    
    # After each epoch, check the validation loss
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        waiting = 0
    else:
        waiting += 1

    if waiting >= patience:
        print("Early stopping triggered - ending training.")
        break

    break

In [3]:



# Initialize variables for early stopping check
best_val_loss = float("inf")
# Number of epochs to wait before stopping training when performance worsens
# already set patience above
waiting = 0

# training with 200 epoch or early stopping
print("*" * 20)
print("*" * 20)
print(f"Training for the total number of epoch {epoch} with batch_size {batch_size} for datasize of {data_size}")
print("*" * 20)
print("*" * 20)
for epoch in range(epoch):
    tensorflow_framework.model.train_loss.reset_states()
    tensorflow_framework.model.train_performance.reset_states()
    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for images, labels in tensorflow_framework.train_ds:
        train_acc, train_loss= tensorflow_framework.fit(images, labels)

    for test_images, test_labels in tensorflow_framework.test_ds:
        test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

    print("Epoch {} - Train Acc: {:.2f} -- Train Loss {} Test Acc {:.2f}  Test Loss {}".format(epoch+1,
                                                                                       train_acc * 100,
                                                                                       train_loss,
                                                                                       test_acc * 100,
                                                                                       test_loss))
    
    # After each epoch, check the validation loss
    if test_loss < best_val_loss:
        best_val_loss = test_loss
        waiting = 0
    else:
        waiting += 1

    if waiting >= patience:
        print("Early stopping triggered - ending training.")
        break





********************
********************
config tensorflow using gpu successfully
********************
********************
==> Preparing data...


2023-07-02 06:23:42.596178: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22301 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:19:00.0, compute capability: 8.6


********************
********************
Training for the total number of epoch 200 with batch_size 128 for datasize of 50000
********************
********************


2023-07-02 06:23:44.834980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [50000,10]
	 [[{{node Placeholder/_1}}]]
2023-07-02 06:23:44.835227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [50000,10]
	 [[{{node Placeholder/_1}}]]
2023-07-02 06:23:48.962951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-02 06:23:49.994582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-

Epoch 1 - Train Acc: 30.67 -- Train Loss 4.157647132873535 Test Acc 36.56  Test Loss 1.8064653873443604
Epoch 2 - Train Acc: 46.36 -- Train Loss 2.94303035736084 Test Acc 45.22  Test Loss 1.5222740173339844
Epoch 3 - Train Acc: 56.67 -- Train Loss 2.2669119834899902 Test Acc 58.00  Test Loss 1.1961127519607544
Epoch 4 - Train Acc: 64.33 -- Train Loss 1.7996292114257812 Test Acc 62.35  Test Loss 1.0761693716049194
Epoch 5 - Train Acc: 69.51 -- Train Loss 1.4914476871490479 Test Acc 57.81  Test Loss 1.244343876838684
Epoch 6 - Train Acc: 73.79 -- Train Loss 1.2730768918991089 Test Acc 62.25  Test Loss 1.0836973190307617
Epoch 7 - Train Acc: 77.22 -- Train Loss 1.1181416511535645 Test Acc 67.88  Test Loss 0.9717263579368591
Epoch 8 - Train Acc: 79.10 -- Train Loss 1.0270195007324219 Test Acc 76.68  Test Loss 0.6771790385246277
Epoch 9 - Train Acc: 80.58 -- Train Loss 0.9676506519317627 Test Acc 69.56  Test Loss 0.9019517302513123
Epoch 10 - Train Acc: 81.66 -- Train Loss 0.930846989154815

In [4]:
# save weights
save_location = "weights2.pkl"
weights = model.get_weights()

with open(save_location, 'wb') as f:
    import pickle
    pickle.dump(weights, f)